In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/ventilator-pressure-prediction/train.csv")
test = pd.read_csv("/kaggle/input/ventilator-pressure-prediction/test.csv")
submission = pd.read_csv("/kaggle/input/ventilator-pressure-prediction/sample_submission.csv")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.corr(method ='pearson')

In [ ]:
train.describe()

In [ ]:
from sklearn.linear_model import LinearRegression
predictors = ['R', 'C', 'time_step', 'u_in', 'u_out']
outcome = 'pressure'
#pressure_lm = LinearRegression()
#pressure_lm.fit(train[predictors], train[outcome])

In [ ]:
scaler = RobustScaler()
xtrain = scaler.fit_transform(xtrain)
xtest = scaler.transform(xtest)
from sklearn.linear_model import LinearRegression
#lin = LinearRegression()
predictors = ['R', 'C', 'time_step', 'u_in', 'u_out']
outcome = 'pressure'
pressure_lm = LinearRegression()
pressure_lm.fit(train[predictors], train[outcome]) 
#lin.fit(X, y)

from sklearn.preprocessing import PolynomialFeatures
 
poly = PolynomialFeatures(degree = 3)
X_poly = poly.fit_transform(train[predictors])
 
poly.fit(X_poly, train[outcome])
lin2 = LinearRegression()
lin2.fit(X_poly, train[outcome])

In [ ]:
#pressure_lm
lin2

In [ ]:
#print(f'Intercept: {pressure_lm.intercept_:.3f}')
#print('Coefficients: ')
#for name, coef in zip(predictors, pressure_lm.coef_):
#    print(f'{name}: {coef}')

print(f'Intercept: {lin2.intercept_:.3f}')
print('Coefficients: ')
for name, coef in zip(predictors, lin2.coef_):
    print(f'{name}: {coef}')

In [ ]:
'''
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
fitted = pressure_lm.predict(train[predictors])
RMSE = np.sqrt(mean_squared_error(train[outcome], fitted))
r2 = r2_score(train[outcome], fitted)
print(f'RMSE: {RMSE:.0f}')
print(f'r2: {r2:.4f}')
'''
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
fitted = lin2.predict(poly.fit_transform(train[predictors]))
RMSE = np.sqrt(mean_squared_error(train[outcome], fitted))
r2 = r2_score(train[outcome], fitted)
print(f'RMSE: {RMSE:.0f}')
print(f'r2: {r2:.4f}')

In [ ]:
#new_fitted = pressure_lm.predict(test[predictors])
new_fitted = lin2.predict(poly.fit_transform(test[predictors]))

In [ ]:
test['id']

In [ ]:
print(test.shape)
print(submission.shape)
print(len(fitted))

In [ ]:
output = pd.DataFrame({'id': test['id'], 'pressure': new_fitted})

In [ ]:
output

In [ ]:
output.to_csv('submission.csv', index=False)

In [ ]:
import statsmodels.api as sm
model = sm.OLS(train[outcome],train[predictors].assign(const=1))
results = model.fit()
results.summary()

In [ ]:
import seaborn as sns
sns.displot(train, x="pressure", kind="kde",bw_adjust=.25)

In [ ]:
train = train.drop(['id', 'breath_id'], axis = 1)

In [ ]:
train.head(n=1)

In [ ]:
from sklearn import preprocessing
d = preprocessing.normalize(train)
scaled_train = pd.DataFrame(d,columns=train.columns)
scaled_train.head()

In [ ]:
pip install keras==2.3.1

In [ ]:
pressure_lm = LinearRegression()
pressure_lm.fit(scaled_train[predictors], scaled_train[outcome])

In [ ]:
print(f'Intercept: {pressure_lm.intercept_:.3f}')
print('Coefficients: ')
for name, coef in zip(predictors, pressure_lm.coef_):
    print(f'{name}: {coef}')

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
fitted = pressure_lm.predict(scaled_train[predictors])
RMSE = np.sqrt(mean_squared_error(scaled_train[outcome], fitted))
r2 = r2_score(scaled_train[outcome], fitted)
print(f'RMSE: {RMSE:.0f}')
print(f'r2: {r2:.4f}')

In [ ]:
new_fitted = pressure_lm.predict(test[predictors])

In [ ]:
output = pd.DataFrame({'id': test['id'], 'pressure': new_fitted})

In [ ]:
output

In [ ]:
output.to_csv('submission.csv', index=False)

In [ ]:
#from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# load dataset
#dataframe = read_csv("housing.csv", delim_whitespace=True, header=None)
#dataset = dataframe.values
# split into input (X) and output (Y) variables
X = train[predictors]
Y = train[outcome]
# define wider model
def wider_model():
	# create model
	model = Sequential()
	model.add(Dense(20, input_dim=5, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=wider_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))